<a href="https://colab.research.google.com/github/pegahl/detect-dyslexia-from-text/blob/main/dyslexia_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install phonetics

  Preparing metadata (setup.py) ... done
  Created wheel for phonetics: filename=phonetics-1.0.5-py2.py3-none-any.whl size=8695 sha256=ac52968722e0ca43b9efde6915a587a687d4e630c72db82a5854bdb0710415cb
  Stored in directory: /root/.cache/pip/wheels/b7/1e/82/80a78c7d1ad7fc6e0af1b4d9009360b251c0e50fe59f046edb
Successfully built phonetics


In [ ]:
pip install levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.2 MB/s eta 0:00:00


In [ ]:
pip install fuzzy

  Preparing metadata (setup.py) ... done
  Created wheel for fuzzy: filename=Fuzzy-1.2.2-cp310-cp310-linux_x86_64.whl size=206407 sha256=c03e7080e4d9e223cf296415437c411daf51b76dc181eaff0dc2b3691760802b
  Stored in directory: /root/.cache/pip/wheels/3e/a7/03/cb9f418321ad60736caa7a86f0077f6ab74b669885c3e03cc1
Successfully built fuzzy


###Phase zero:

####I extracted some features from the list of common diagnostic errors of dyslexic children based on the needs of the project. from 15 common errors, I chose 6 errors.
---
Categorized list of the common types of errors:
1. Phonetic Spelling
2. Homophone Confusion
3. Letter Transposition
4. Letter Omission
5. Punctuation Omission
6. Phonetic Substitution
7. Grapheme-Phoneme Confusion
8. Vowel Substitution
9. Double Consonant Confusion
10. Morphological Errors
11. Visual Similarity Confusion
12. Word Boundary Errors
13. Dysgraphia-Related Errors (Handwriting Difficulties)
14. Sequential Memory Issues
15. Semantic Confusion
---
Six common and important types of language errors:
1. Letter Omission
2. Letter Transposition
3. Homophone Confusion
4. Vowel Substitution
5. Extract Punctuation Omission
6. Grapheme-Phoneme Confusion




In [ ]:
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('cmudict')
import re
import difflib
from nltk.corpus import words
from nltk.corpus import cmudict
from nltk.util import ngrams
from collections import defaultdict
from Levenshtein import distance as levenshtein_distance
from nltk.tokenize import word_tokenize
import phonetics
from fuzzywuzzy import process
from itertools import permutations
import string
import fuzzy


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
# Download the words corpus if not already downloaded
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
# Get the list of correct words in English
correct_words = set([word for word in words.words() if word.isalpha() and len(word) > 1])
word_list = set(words.words())

### ok but not perfect

In [ ]:
def detect_spelling_errors(text, threshold=80):
    # Tokenize the input text
    tokens = word_tokenize(text)

    spelling_errors = {}

    for token in tokens:
        lower_token = token.lower()

        # Skip non-alphabetic tokens and very short tokens
        if not lower_token.isalpha() or len(lower_token) < 2:
            continue

        # Find the closest match using Levenshtein Distance
        best_match = process.extractOne(lower_token, correct_words)

        # If similarity score is below the threshold, we consider it an error
        if best_match and best_match[1] < threshold:
            spelling_errors[token] = best_match[0]

    return spelling_errors

# Example text from the child
text = """Tim was playng in his yard wen he herd a nosie. It was a littel puppy!
The puppy was hideing and didnt have a coller. Tim thot the puppy was lost so he toke him home.
He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy."""

# Detect spelling errors
spelling_errors = detect_spelling_errors(text, threshold=85)
print("Spelling Errors Detected:")
for error, suggestion in spelling_errors.items():
    print(f"{error} -> {suggestion}")


Spelling Errors Detected:


In [ ]:
#extra stuff
def phonetic_similarity(word1, word2):
    phonemes1 = set(ngrams(word1, 2))
    phonemes2 = set(ngrams(word2, 2))

    intersection = phonemes1.intersection(phonemes2)
    union = phonemes1.union(phonemes2)

    return len(intersection) / len(union)


In [ ]:
# import nltk
# from nltk.corpus import cmudict
# from collections import defaultdict

# # Download required NLTK data
# nltk.download('cmudict')
# nltk.download('punkt')

# def load_phonemes():
#     d = cmudict.dict()
#     return {word.lower(): [tuple(p[0] for p in pron) for pron in d[word.lower()]][:5]}

# def phonetic_mismatch(word):
#     phonemes = load_phonemes()[word.lower()]
#     known_words = set(word.lower() for word in load_phonemes().keys())

#     # Check if the word exists in the dictionary
#     if word.lower() not in known_words:
#         return True

#     # Check if the phoneme representation matches the word
#     if len(phonemes) == 0:
#         return True

#     # Check for unexpected phonemes
#     unexpected_phones = set(phonemes).difference(set(load_phonemes().values()))
#     if unexpected_phones:
#         return True

#     return False

# def detect_phonetic_spelling(text, threshold=0.7):
#     phonemes = load_phonemes()
#     words = nltk.word_tokenize(text.lower())
#     mismatches = []

#     for i, word in enumerate(words):
#         if phonetic_mismatch(word):
#             mismatches.append((i, word))

#     return mismatches

# # Example usage
# text = "The quick brown fox jumps over the lazy dog."
# mismatches = detect_phonetic_spelling(text)

# for index, word in mismatches:
#     print(f"Possible phonetic spelling issue: '{word}' at position {index}")


In [ ]:
#simple exmple:
sentence = """At eight o'clock on Thursday morning
... Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentence)
tokens

['At',
 'eight',
 "o'clock",
 'on',
 'Thursday',
 'morning',
 '...',
 'Arthur',
 'did',
 "n't",
 'feel',
 'very',
 'good',
 '.']

In [ ]:
# Preload words dictionary for comparison
nltk_words = set(words.words())


**Load Dyslexia Data**

In [ ]:
#sample text
text="""Tim was playng in his yard wen he herd a nosie. It was a littel puppy! The puppy was hideing and didnt have a coller.
Tim thot the puppy was lost so he toke him home.
He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy.
"""

**Load Normal data**

#0. Extract Phonetic Spelling

*  Description: Writing words as they sound phonetically, which often leads to incorrect spelling.
*   Example: wut instead of what





## Finding best algorithm for Phonetic spelling

**1. Soundex Algorithm**

In [ ]:

# # Get the list of correct words in English and filter out too short or non-alphabetic words
# correct_words = set([word for word in words.words() if word.isalpha() and len(word) > 1])

# # Create a dictionary of correct words with their Soundex codes
# correct_words_phonetic = {word: phonetics.soundex(word) for word in correct_words}

# def detect_phonetic_spelling(text):
#     # Tokenize the input text
#     tokens = word_tokenize(text)

#     phonetic_errors = {}

#     for token in tokens:
#         lower_token = token.lower()

#         # Skip non-alphabetic tokens and very short tokens
#         if not lower_token.isalpha() or len(lower_token) < 2:
#             continue

#         token_soundex = phonetics.soundex(lower_token)

#         # If the word is not in the correct words list, check phonetically similar words
#         if lower_token not in correct_words:
#             # Find words with matching Soundex codes
#             similar_words = [word for word, soundex_code in correct_words_phonetic.items() if soundex_code == token_soundex]

#             if similar_words:
#                 phonetic_errors[token] = similar_words[0]  # Choose the first match

#     return phonetic_errors

# # Example text from the child
# text1 = """Tim was playng in his yard wen he herd a nosie. It was a littel puppy!
# The puppy was hideing and didnt have a coller. Tim thot the puppy was lost so he toke him home.
# He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy."""

# # Detect phonetic spelling errors
# phonetic_errors = detect_phonetic_spelling(text)
# print("Phonetic Spelling Errors Detected:")
# for error, suggestion in phonetic_errors.items():
#     print(f"{error} -> {suggestion}")


**2. Levenshtein Distance**

In [ ]:
# # Function to detect Phonetic Spelling Errors (Levenshtein Distance)
# def detect_phonetic_spelling(word, dictionary=nltk_words, threshold=2):
#     closest_words = difflib.get_close_matches(word, dictionary, n=3)
#     for correct_word in closest_words:
#         if levenshtein_distance(word, correct_word) <= threshold:
#             return f"Phonetic spelling error: '{word}' is close to '{correct_word}'"
#     return None


**3.**

#1. Extract Letter Omission

*  Description: Leaving out necessary letters within a word.

*  Example: frend instead of friend





we have to ignore Letter Omission for this phase, cause we have a conflict between different errors. such as in our first sample text we detect "thot" as a letter omission but actually "thot" was "that" and we had Vowel Substitution! So I think that it's ok to ignore this error for now.

In [ ]:
# Sample text
text = """
Tim was playng in his yard wen he herd a nosie. It was a littel puppy! The puppy was hideing and didnt have a coller.
Tim thot the puppy was lost so he toke him home.
He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy.
"""
#new code!
def detect_letter_omission(word):
  # A list of valid English words
  word_list = set(words.words())

  # Tokenize the text
  tokens = nltk.word_tokenize(text.lower())

  # Function to detect letter omission errors
  def detect_omissions(word):
      # Check if the word is in the dictionary
      if word in word_list:
          return None  # Word is correct, no need to check for omissions

      alphabet = string.ascii_lowercase
      candidates = set()

      # Try adding a letter at each position in the word
      for i in range(len(word) + 1):
          for letter in alphabet:
              candidate_word = word[:i] + letter + word[i:]
              if candidate_word in word_list:
                  candidates.add(candidate_word)

      if candidates:
          return candidates
      else:
          return None

  # Check each word in the text for omissions
  omission_errors = {}
  for token in tokens:
      omissions = detect_omissions(token)
      if omissions:
          omission_errors[token] = omissions

  # Print the omission errors found
  if omission_errors:
      print("Omission Errors Found:")
      for original, corrections in omission_errors.items():
          print(f"'{original}' might have a letter omitted. Suggestions: {corrections}")
      print("\n")
  else:
      print("No omission errors found.")
  return None

In [ ]:
detect_letter_omission(text)

Omission Errors Found:
'tim' might have a letter omitted. Suggestions: {'trim', 'time', 'stim'}
'nosie' might have a letter omitted. Suggestions: {'nosine'}
'hideing' might have a letter omitted. Suggestions: {'hideling'}
'coller' might have a letter omitted. Suggestions: {'collier', 'choller', 'collery'}
'thot' might have a letter omitted. Suggestions: {'thowt', 'thoft', 'thort'}
'calld' might have a letter omitted. Suggestions: {'callid'}
'max' might have a letter omitted. Suggestions: {'maux'}
'sed' might have a letter omitted. Suggestions: {'seed', 'used', 'sned', 'sped', 'send', 'sled', 'shed'}




#2. Extract Letter Transposition

*  Description: Reversing the order of letters in a word.
* Example: gril instead of girl






In [ ]:
def detect_letter_transpositions(word):
  # Tokenize the text
  tokens = nltk.word_tokenize(text.lower())

  # Function to detect letter transpositions
  def detect_transpositions(word):
      # Check if word is in the dictionary
      if word in word_list:
          return None  # Word is correct, no need to check for transposition

      # Generate all possible transpositions by swapping adjacent letters
      transposed_candidates = set()
      for i in range(len(word) - 1):
          transposed_word = list(word)
          # Swap adjacent letters
          transposed_word[i], transposed_word[i + 1] = transposed_word[i + 1], transposed_word[i]
          transposed_candidates.add("".join(transposed_word))

      # Check if any transposed candidates are valid words
      valid_transpositions = [w for w in transposed_candidates if w in word_list]

      if valid_transpositions:
          return valid_transpositions
      else:
          return None

  # Check each word in the text for transpositions
  transposition_errors = {}
  for token in tokens:
      transpositions = detect_transpositions(token)
      if transpositions:
          transposition_errors[token] = transpositions

  # Print the transposition errors found
  if transposition_errors:
      print("Transposition Errors Found:")
      for original, corrections in transposition_errors.items():
        # .join(my_list)
          print(f"Letter transposition detected: '{original}' might be",*corrections)
          # print(f"'{original}' might be a transposition. Suggestions: {corrections}")
      print("\n")
  else:
      print("No transposition errors found.")
  return

In [ ]:
detect_letter_transpositions(text)

Transposition Errors Found:
Letter transposition detected: 'nosie' might be noise
Letter transposition detected: 'littel' might be little




#3. Extract Homophone Confusion


*  Description: Using the wrong word that sounds the same but is spelled differently.
* Example: there instead of their






This code can't detect the correct form of the word! code Just sugested that we might be have error (for detect the complete error we need to know that this error is part of grammeral errors and we need to use another algorithms, so I ignore this error for this phase.
I will implement this code for phase 2(the grammeral errors))

In [ ]:
# Sample text with homophone confusion
text = """
Tim was playing in his yard when he heard a noise. It was a little puppy! The puppy was hiding and didn't have a collar.
Tim thought the puppy was lost, so he took him home.
He called the puppy Max. Tim’s parents said they had to find the owner, so they went to the animal shelter, but no one knew the puppy.
In school, Tim loved physics and chemistry.
He left his book over there, but he meant to take it with him to their house.
"""
def detect_homophone_confusion(word):


  # Tokenize the text
  tokens = nltk.word_tokenize(text.lower())

  # A dictionary of common homophones
  homophones = {
      "there": {"there", "their", "they're"},
      "their": {"there", "their", "they're"},
      "they're": {"there", "their", "they're"},
      "your": {"your", "you're"},
      "you're": {"your", "you're"},
      "to": {"to", "too", "two"},
      "too": {"to", "too", "two"},
      "two": {"to", "too", "two"},
      "its": {"its", "it's"},
      "it's": {"its", "it's"},
      # Add more homophone sets as needed
  }

  # Function to detect homophone confusion
  def detect_homophone_confusion(tokens):
      homophone_errors = {}

      for token in tokens:
          # If the token is in the homophone dictionary, check for homophone confusion
          if token in homophones:
              homophone_set = homophones[token]
              # Flagging as a possible confusion, since it's part of a homophone group
              homophone_errors[token] = homophone_set

      return homophone_errors

  # Check the text for homophone confusion
  homophone_confusions = detect_homophone_confusion(tokens)

  # Print the homophone confusion errors found
  if homophone_confusions:
      print("Homophone Confusion Errors Found:")
      for word, homophone_set in homophone_confusions.items():
          print(f"'{word}' is part of the homophone set: {homophone_set}. Possible confusion.")
      print("\n")
  else:
      print("No homophone confusion errors found.")
  return None

In [ ]:
detect_homophone_confusion(text)

Homophone Confusion Errors Found:
'to' is part of the homophone set: {'too', 'two', 'to'}. Possible confusion.
'there' is part of the homophone set: {"they're", 'there', 'their'}. Possible confusion.
'their' is part of the homophone set: {"they're", 'there', 'their'}. Possible confusion.




#4. Extract Vowel Substitution

*  Description: Incorrectly substituting vowels in words.
* Example: hat instead of hit





In [ ]:
# Sample text
text = """
Tim was playng in his yard wen he herd a nosie. It was a littel puppy! The puppy was hideing and didnt have a coller.
Tim thot the puppy was lost so he toke him home.
He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy.
"""
def detect_vowel_substitution(word):
  # Define vowels for substitution
  vowels = "aeiou"



  # Tokenize the text
  tokens = nltk.word_tokenize(text.lower())

  # Function to detect vowel substitution errors
  def detect_vowel_substitution(word):
      # Check if the word is in the dictionary
      if word in word_list:
          return None  # Word is correct, no need to check for vowel substitution

      substitution_candidates = set()

      # Replace each vowel in the word with other vowels
      for i, char in enumerate(word):
          if char in vowels:
              for vowel in vowels:
                  if char != vowel:  # Only substitute with a different vowel
                      substituted_word = word[:i] + vowel + word[i+1:]
                      if substituted_word in word_list:
                          substitution_candidates.add(substituted_word)

      if substitution_candidates:
          return substitution_candidates
      else:
          return None

  # Check each word in the text for vowel substitutions
  vowel_substitution_errors = {}
  for token in tokens:
      substitutions = detect_vowel_substitution(token)
      if substitutions:
          vowel_substitution_errors[token] = substitutions

  # Print the vowel substitution errors found
  if vowel_substitution_errors:
      print("Vowel Substitution Errors Found:")
      for original, corrections in vowel_substitution_errors.items():
          print(f"'{original}' might have a vowel substitution. Suggestions: {corrections}")
      print("\n")
  else:
      print("No vowel substitution errors found.")
  return None

In [ ]:
detect_vowel_substitution(text)

Vowel Substitution Errors Found:
'tim' might have a vowel substitution. Suggestions: {'tum', 'tam'}
'coller' might have a vowel substitution. Suggestions: {'culler', 'caller', 'collar'}
'thot' might have a vowel substitution. Suggestions: {'that'}
'max' might have a vowel substitution. Suggestions: {'mix', 'mux'}
'sed' might have a vowel substitution. Suggestions: {'sad', 'sod', 'sud'}
'animel' might have a vowel substitution. Suggestions: {'animal'}
'sheltur' might have a vowel substitution. Suggestions: {'shelter'}




#5. Extract Punctuation Omission


*  Description: Missing punctuation, especially in contractions or possessives.
* Example: cant instead of can't







In [ ]:
# List of common contractions
contractions = {"can't": 'cannot', "don't": 'do not', "didn't": 'did not', "won't": 'will not', "couldn't":'could not', "wasn't":'was not'}


# Function to detect Punctuation Omission (e.g., "cant" instead of "can't")
def detect_punctuation_omission(word):
    if word in contractions.keys():
        return None  # Word is a valid contraction
    for contraction, full_word in contractions.items():
        if word == contraction.replace("'", ""):
            return f"Punctuation omission detected:\n'{word}' might be '{contraction}'"

    return None


#6. Extract Grapheme-Phoneme Confusion


*  Description: Difficulty matching sounds (phonemes) to the correct letters or letter combinations (graphemes).
* Example: fiziks instead of physics






In [ ]:
pip install metaphone


  Preparing metadata (setup.py) ... done
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13900 sha256=80ade2252937e97f1bbbdd5eba10afcaddc4ecbc88bd49b6f5aa75a9577ef718
  Stored in directory: /root/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built metaphone


In [ ]:
import nltk
from nltk.corpus import words
from nltk.metrics import edit_distance
from metaphone import doublemetaphone

In [ ]:
# A list of valid English words
word_list = set(words.words())

# Sample text
text = """
fizics Tim was playng in his yard wen he herd a nosie. It was a littel puppy! The puppy was hideing and didnt have a coller.
Tim thot the puppy was lost so he toke him home.
He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy.
In school, Tim loved fiziks and chemestry.
"""

# Tokenize the text
tokens = nltk.word_tokenize(text.lower())
def detect_grapheme_phoneme(word):
  # Function to detect grapheme-phoneme confusion using Metaphone
  def detect_grapheme_phoneme_confusion(word):
      # Check if the word is in the dictionary
      if word in word_list:
          return None  # Word is correct, no need to check for confusion

      # Get the Metaphone encoding of the input word
      word_metaphone = doublemetaphone(word)[0]

      # Find valid dictionary words with the same Metaphone code
      candidates = [w for w in word_list if doublemetaphone(w)[0] == word_metaphone]

      if candidates:
          return candidates[0]  # Return the first matching word
      else:
          return None

  # Check each word in the text for grapheme-phoneme confusion
  grapheme_phoneme_errors = {}
  for token in tokens:
      correction = detect_grapheme_phoneme_confusion(token)
      if correction:
          grapheme_phoneme_errors[token] = correction

  # Print the grapheme-phoneme confusion errors found
  if grapheme_phoneme_errors:
      print("Grapheme-Phoneme Confusion Errors Found:")
      for original, correction in grapheme_phoneme_errors.items():
          print(f"'{original}' might be a phonetic confusion. Suggested correction: '{correction}'")
  else:
      print("No grapheme-phoneme confusion errors found.")
  return None


In [ ]:
# Tokenize the text
tokens = nltk.word_tokenize(text.lower())

# Function to detect grapheme-phoneme confusion using Metaphone
def detect_grapheme_phoneme_confusion(word):
    # If the word is already a valid word, no correction is needed
    if word in word_list:
        return None

    # Get the Metaphone encoding of the input word (primary encoding)
    word_metaphone = doublemetaphone(word)[0]

    # Find valid words with the same Metaphone code
    for w in word_list:
        if doublemetaphone(w)[0] == word_metaphone:
            return w  # Return the first valid matching word

    return None

# Check each word in the text for grapheme-phoneme confusion
grapheme_phoneme_errors = {}
for token in tokens:
    correction = detect_grapheme_phoneme_confusion(token)
    if correction:
        grapheme_phoneme_errors[token] = correction

# Print the grapheme-phoneme confusion errors found
if grapheme_phoneme_errors:
    print("Grapheme-Phoneme Confusion Errors Found:")
    for original, correction in grapheme_phoneme_errors.items():
        print(f"'{original}' might be a phonetic confusion. Suggested correction: '{correction}'")
else:
    print("No grapheme-phoneme confusion errors found.")


Grapheme-Phoneme Confusion Errors Found:
'fizics' might be a phonetic confusion. Suggested correction: 'Phascaceae'
'tim' might be a phonetic confusion. Suggested correction: 'taum'
'playng' might be a phonetic confusion. Suggested correction: 'blink'
'nosie' might be a phonetic confusion. Suggested correction: 'Nosu'
'.' might be a phonetic confusion. Suggested correction: 'W'
'littel' might be a phonetic confusion. Suggested correction: 'lootiewallah'
'!' might be a phonetic confusion. Suggested correction: 'W'
'hideing' might be a phonetic confusion. Suggested correction: 'hatting'
'coller' might be a phonetic confusion. Suggested correction: 'glower'
'thot' might be a phonetic confusion. Suggested correction: 'thwaite'
'calld' might be a phonetic confusion. Suggested correction: 'collate'
'max' might be a phonetic confusion. Suggested correction: 'mycose'
'’' might be a phonetic confusion. Suggested correction: 'W'
'parnts' might be a phonetic confusion. Suggested correction: 'bran

In [ ]:
# #with Soundex(doesn't work for now)
# import fuzzy
# # A list of valid English words
# word_list = set(words.words())

# # Initialize Soundex
# soundex = fuzzy.Soundex(4)

# # Sample text
# text = """
# fizics Tim was playng in his yard wen he herd a nosie. It was a littel puppy! The puppy was hideing and didnt have a coller.
# Tim thot the puppy was lost so he toke him home.
# He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy.
# In school, Tim loved fiziks and chemestry.
# """

# # Tokenize the text
# tokens = nltk.word_tokenize(text.lower())

# # Function to detect grapheme-phoneme confusion
# def detect_grapheme_phoneme_confusion(word):
#     # Check if the word is in the dictionary
#     if word in word_list:
#         return None  # Word is correct, no need to check for confusion

#     # Get the Soundex code of the input word
#     word_soundex = soundex(word)

#     # Find valid dictionary words with the same Soundex code
#     candidates = [w for w in word_list if soundex(w) == word_soundex]

#     if candidates:
#         return candidates[0]  # Return the first matching word
#     else:
#         return None

# # Check each word in the text for grapheme-phoneme confusion
# grapheme_phoneme_errors = {}
# for token in tokens:
#     correction = detect_grapheme_phoneme_confusion(token)
#     if correction:
#         grapheme_phoneme_errors[token] = correction

# # Print the grapheme-phoneme confusion errors found
# if grapheme_phoneme_errors:
#     print("Grapheme-Phoneme Confusion Errors Found:")
#     for original, correction in grapheme_phoneme_errors.items():
#         print(f"'{original}' might be a phonetic confusion. Suggested correction: '{correction}'")
# else:
#     print("No grapheme-phoneme confusion errors found.")


## not finished!

*double consonant confusion*

In [ ]:
import nltk

# Download words corpus if necessary
nltk.download('words')

# List of English words from NLTK
word_list = set(nltk.corpus.words.words())

# Sample text containing possible double consonant confusion
text = """
Tim was stoping by the store to get some supplies. He was also planing a trip for the weekend.
The puppy was running around and jumping everywhere.
"""

# Tokenize the text
tokens = nltk.word_tokenize(text.lower())

# Function to detect double consonant confusion
def detect_double_consonant_confusion(word):
    # If the word is valid (in dictionary), return None
    if word in word_list:
        return None

    # Check if the word is missing a double consonant
    for i in range(1, len(word)):
        # If there is a consonant followed by the same consonant (i.e., double consonant)
        if word[i] == word[i - 1] and word[i] in "bcdfghjklmnpqrstvwxyz":
            return None  # This word already has the double consonant, no confusion

    # Now check for possible confusion with valid words that contain double consonants
    for w in word_list:
        if len(w) == len(word) + 1 and w.startswith(word[:-1]) and w[-1] == w[-2]:
            return w  # Return the correct word with the double consonant

    return None

# Check each word in the text for double consonant confusion
double_consonant_errors = {}
for token in tokens:
    correction = detect_double_consonant_confusion(token)
    if correction:
        double_consonant_errors[token] = correction

# Print the double consonant confusion errors found
if double_consonant_errors:
    print("Double Consonant Confusion Errors Found:")
    for original, correction in double_consonant_errors.items():
        print(f"'{original}' might be missing a double consonant. Suggested correction: '{correction}'")
else:
    print("No double consonant confusion errors found.")


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Double Consonant Confusion Errors Found:
'tim' might be missing a double consonant. Suggested correction: 'till'
'.' might be missing a double consonant. Suggested correction: 'aa'


In [ ]:
import nltk
import re

# Download words corpus if necessary
nltk.download('words')

# List of English words from NLTK
word_list = set(nltk.corpus.words.words())

# Sample text containing possible double consonant confusion
text = """
Tim was stoping by the store to get some supplies. He was also planing a trip for the weekend.
The puppy was running around and jumping everywhere.
"""

# Tokenize the text
tokens = nltk.word_tokenize(text.lower())

# Regular expression to identify words that may need double consonants before 'ing' or 'ed'
double_consonant_pattern = re.compile(r'([bcdfghjklmnpqrstvwxyz])([aeiou])([bcdfghjklmnpqrstvwxyz])(?:ing|ed)$')

# Function to detect double consonant confusion
def detect_double_consonant_confusion(word):
    if word in word_list:
        return None

    match = double_consonant_pattern.match(word)
    if match:
        # Extract consonants and vowel parts
        first_consonant, vowel, last_consonant = match.groups()

        # Form the correct double consonant word
        correct_word = word.replace(last_consonant, last_consonant * 2)

        # Check if the corrected word exists in the dictionary
        if correct_word in word_list:
            return correct_word

    return None

# Check each word in the text for double consonant confusion
double_consonant_errors = {}
for token in tokens:
    correction = detect_double_consonant_confusion(token)
    if correction:
        double_consonant_errors[token] = correction

# Print the double consonant confusion errors found
if double_consonant_errors:
    print("Double Consonant Confusion Errors Found:")
    for original, correction in double_consonant_errors.items():
        print(f"'{original}' might be missing a double consonant. Suggested correction: '{correction}'")
else:
    print("No double consonant confusion errors found.")


No double consonant confusion errors found.


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import nltk
import re

In [ ]:


# Download words corpus if necessary
nltk.download('words')

# List of valid English words from NLTK
word_list = set(nltk.corpus.words.words())

# Sample text containing possible double consonant confusion
text = """
Tim was stoping by the store to get some supplies. He was also planing a trip for the weekend.
The puppy was running around and jumping everywhere.
"""

# Tokenize the text
tokens = nltk.word_tokenize(text.lower())

# Regular expression to match words with a single consonant before suffixes like "ing" or "ed"
double_consonant_pattern = re.compile(r'([bcdfghjklmnpqrstvwxyz])([aeiou])([bcdfghjklmnpqrstvwxyz])(ing|ed|er)$')

# Function to detect double consonant confusion
def detect_double_consonant_confusion(word):
    # Check if the word is already valid
    if word in word_list:
        return None

    # Check for potential missing double consonants using regex
    match = double_consonant_pattern.search(word)
    if match:
        first_consonant, vowel, last_consonant, suffix = match.groups()

        # Form the correct double consonant version of the word
        corrected_word = word.replace(last_consonant + suffix, last_consonant * 2 + suffix)

        # Check if the corrected word is in the dictionary
        if corrected_word in word_list:
            return corrected_word

    return None

# Check each word in the text for double consonant confusion
double_consonant_errors = {}
for token in tokens:
    correction = detect_double_consonant_confusion(token)
    if correction:
        double_consonant_errors[token] = correction

# Print the double consonant confusion errors found
if double_consonant_errors:
    print("Double Consonant Confusion Errors Found:")
    for original, correction in double_consonant_errors.items():
        print(f"'{original}' might be missing a double consonant. Suggested correction: '{correction}'")
else:
    print("No double consonant confusion errors found.")


No double consonant confusion errors found.


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


# 7.Extreact visual similarity errors

In [ ]:


# Download the words corpus if necessary
nltk.download('words')

# List of valid English words from NLTK
word_list = set(nltk.corpus.words.words())

# Sample text containing possible visual similarity confusion
text = """
Tim saw a qen outside the window. He also mistook his pen for a qencil. The little puppy was barking at the big bog. lmportant things
"""

# Tokenize the text
tokens = nltk.word_tokenize(text.lower())
def detect_visual_similarity_confusion(word):
  # Dictionary of visually similar letters
  visual_similarity = {
      'b': 'd', 'd': 'b',
      'p': 'q', 'q': 'p',
      'i': 'l', 'l': 'i',
      'v': 'w', 'w': 'v',
      'c': 'e', 'e': 'c'
  }

  # Function to detect visual similarity confusion
  def detect(word):
      if word in word_list:
          return None, None

      # List to store possible corrections
      possible_corrections = []

      # Try replacing each letter with its visually similar counterpart
      for i, letter in enumerate(word):
          if letter in visual_similarity:
              # Substitute with the visually similar letter
              new_word = word[:i] + visual_similarity[letter] + word[i+1:]

              # Check if the new word exists in the dictionary
              if new_word in word_list:
                  possible_corrections.append(new_word)

      if possible_corrections:
          # Return the first valid correction and the error type
          return possible_corrections[0], "Visual similarity confusion"

      return None, None

  # Check each word in the text for visual similarity confusion
  visual_similarity_errors = {}
  for token in tokens:
      correction, error_type = detect(token)
      if correction:
          visual_similarity_errors[token] = {
              'correct_word': correction,
              'error_type': error_type
          }

  # Print the visual similarity confusion errors found with details
  if visual_similarity_errors:
      print("Visual Similarity Confusion Errors Found:")
      for incorrect_word, details in visual_similarity_errors.items():
          print(f"Incorrect Word: '{incorrect_word}'")
          print(f"Correct Word: '{details['correct_word']}'")
          print(f"Error Type: {details['error_type']}")
          print("---")

  else:
      print("No visual similarity confusion errors found.")

  print("\n")
  return None


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
detect_visual_similarity_confusion(text)

Visual Similarity Confusion Errors Found:
Incorrect Word: 'qen'
Correct Word: 'pen'
Error Type: Visual similarity confusion
---
Incorrect Word: 'qencil'
Correct Word: 'pencil'
Error Type: Visual similarity confusion
---
Incorrect Word: 'lmportant'
Correct Word: 'important'
Error Type: Visual similarity confusion
---


# 8. Ectract word_boundary_errors

In [ ]:
common_word_boundary_errors = {
    # Commonly combined words
    "alot": ["a", "lot"],
    "infact": ["in", "fact"],
    "theres": ["there", "is"],
    "heres": ["here", "is"],
    "its": ["it", "is"],  # When incorrectly combined without apostrophe
    "lets": ["let", "us"],
    "whos": ["who", "is"],  # When not used as possessive
    "im": ["i", "am"],
    "youre": ["you", "are"],
    "theyre": ["they", "are"],
    "were": ["we", "are"],  # In case it's mistaken with "we're"

    # Other cases often confused or mistakenly joined
    "inorder": ["in", "order"],
    "thankyou": ["thank", "you"],
    "eachother": ["each", "other"],
    "nomatter": ["no", "matter"],
    "aswell": ["as", "well"],
    "allright": ["all", "right"],  # Variation of alright
    "anymore": ["any", "more"],  # For certain dialects
    "anyway": ["any", "way"],  # Some dialects use "anyways"
    "nevermind": ["never", "mind"],
    "alotof": ["a", "lot", "of"],  # When students combine the phrase into one word
    "kindof": ["kind", "of"],  # Informal writing mistake
    "sortof": ["sort", "of"],
    "mustnt": ["must", "not"],  # Variations of negations with contractions
    "havent": ["have", "not"],
    "hasnt": ["has", "not"],
    "whered": ["where", "did"],  # Common question forms
    "howd": ["how", "did"],
    "whyd": ["why", "did"],
    "whats": ["what", "is"],  # When improperly contracted
    "gonna": ["going", "to"],  # Informal but might be confused as one word
    "wanna": ["want", "to"],
    "hafta": ["have", "to"],  # Informal pronunciation but often written together
}


In [ ]:


# Download the words corpus if necessary
nltk.download('words')

# List of valid English words from NLTK
word_list = set(nltk.corpus.words.words())

# Sample text containing possible word boundary errors
text = """
Tim thought alot about the puppy. He didnt know if infact the puppy belonged to someone.
He said it was alright and went outside to see if theres anyone around. He thought alotof people would notice. He also said thankyou to the people helping.
"""

# Tokenize the text
tokens = nltk.word_tokenize(text.lower())
def detect_word_boundary_errors(word):
  # Function to detect word boundary errors
  def detect_word_b(word):
      if word in common_word_boundary_errors:
          correct_split = common_word_boundary_errors[word]
          # Check if all parts exist in the word list
          if all(part in word_list for part in correct_split):
              return ' '.join(correct_split), "Word boundary error"

      return None, None

  # Check each word in the text for word boundary errors
  word_boundary_errors = {}
  for token in tokens:
      correction, error_type = detect_word_b(token)
      if correction:
          word_boundary_errors[token] = {
              'correct_word': correction,
              'error_type': error_type
          }

  # Print the word boundary confusion errors found with details
  if word_boundary_errors:
      print("Word Boundary Errors Found:")
      for incorrect_word, details in word_boundary_errors.items():
          print(f"Incorrect Word: '{incorrect_word}'")
          print(f"Correct Word: '{details['correct_word']}'")
          print(f"Error Type: {details['error_type']}")
          print("---")
  else:
      print("No word boundary errors found.")
  print("\n")
  return None

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
detect_word_boundary_errors(text)

Word Boundary Errors Found:
Incorrect Word: 'alot'
Correct Word: 'a lot'
Error Type: Word boundary error
---
Incorrect Word: 'infact'
Correct Word: 'in fact'
Error Type: Word boundary error
---
Incorrect Word: 'theres'
Correct Word: 'there is'
Error Type: Word boundary error
---
Incorrect Word: 'alotof'
Correct Word: 'a lot of'
Error Type: Word boundary error
---
Incorrect Word: 'thankyou'
Correct Word: 'thank you'
Error Type: Word boundary error
---


# Final Step:

"""One Saturday morning, Tim was playing in his backyard when he heard a soft whimper. He looked around and saw a small, scared puppy hiding behind a bush. The puppy didn’t have a collar, so Tim thought it must be lost. He carefully walked up to the puppy, talking softly so it wouldn’t be scared. The puppy wagged its tail and slowly came out. Tim decided to take it home and named it Max.
When Tim’s parents saw the puppy, they said he could keep it for now, but they had to find the owner first. They took Max to the animal shelter to check if anyone had reported him missing, but no one had. Tim’s family put up posters around the neighborhood, hoping someone would come forward. Days went by, and no one claimed Max, so Tim kept playing with him every day. He loved Max and was excited to have a dog.
But one day, when Tim was at the park with Max, a little girl ran up to him. She looked at Max and shouted, "That’s my dog!" Tim felt sad. The girl told him that Max’s real name was Buddy, and he had run away from home about a week ago. Even though Tim didn’t want to let Max go, he knew it was the right thing to do. So, the next day, Tim and his parents took Max to the girl’s house, and Tim said goodbye.
It was hard to say goodbye, but Tim was proud that he did the right thing. A few days later, Tim’s parents surprised him by adopting a new puppy from the animal shelter. Tim named the new dog Lucky, and even though he missed Max, he was happy to have another dog to play with.
"""


In [ ]:
# Sample text
text =  """
Tim was playng in his yard wen he herd a nosie. It was a littel puppy! The puppy was hideing and didnt have a coller.
Tim thot the puppy was lost so he toke him home.
He calld the puppy Max. Tim’s parnts sed they had to find the ower so they went to the animel sheltur but no one knew the puppy. he saw a qen! outside the window.
He also mistook his pen for a qencil and it was lmportant mistake!
Tim thought alot about the puppy. He didnt know if infact the puppy belonged to someone.
He said it was alright and went outside to see if theres anyone around. He thought alotof people would notice. He also said thankyou to the people helping.
"""
tokens = nltk.word_tokenize(text.lower())

In [ ]:

def detect_errors(text):
    words_list = re.findall(r'\b\w+\b', text.lower())  # Basic tokenization
    detect_letter_transpositions(text)
    detect_vowel_substitution(text)
    detect_letter_omission(text)
    detect_homophone_confusion(text)
    detect_visual_similarity_confusion(text)
    detect_word_boundary_errors(text)
    for word in words_list:
        error_checks = [
            detect_punctuation_omission(word),
            # detect_phonetic_spelling(word),
        ]
        for error in error_checks:
            if error:
                print(error)



In [ ]:
detect_errors(text)


Transposition Errors Found:
Letter transposition detected: 'nosie' might be noise
Letter transposition detected: 'littel' might be little
Letter transposition detected: 'alot' might be alto


Vowel Substitution Errors Found:
'tim' might have a vowel substitution. Suggestions: {'tum', 'tam'}
'coller' might have a vowel substitution. Suggestions: {'culler', 'caller', 'collar'}
'thot' might have a vowel substitution. Suggestions: {'that'}
'max' might have a vowel substitution. Suggestions: {'mix', 'mux'}
'sed' might have a vowel substitution. Suggestions: {'sad', 'sod', 'sud'}
'animel' might have a vowel substitution. Suggestions: {'animal'}
'sheltur' might have a vowel substitution. Suggestions: {'shelter'}
'alot' might have a vowel substitution. Suggestions: {'ilot', 'alit'}
'infact' might have a vowel substitution. Suggestions: {'infect', 'unfact'}


Omission Errors Found:
'tim' might have a letter omitted. Suggestions: {'trim', 'time', 'stim'}
'nosie' might have a letter omitted. Sugg

In [ ]:
#normal kid
text1="""One Saturday morning, Tim was playing in his backyard when he heard a soft whimper. He looked around and saw a small, scared puppy hiding behind a bush. The puppy didn’t have a collar, so Tim thought it must be lost. He carefully walked up to the puppy, talking softly so it wouldn’t be scared. The puppy wagged its tail and slowly came out. Tim decided to take it home and named it Max.
When Tim’s parents saw the puppy, they said he could keep it for now, but they had to find the owner first. They took Max to the animal shelter to check if anyone had reported him missing, but no one had. Tim’s family put up posters around the neighborhood, hoping someone would come forward. Days went by, and no one claimed Max, so Tim kept playing with him every day. He loved Max and was excited to have a dog.
But one day, when Tim was at the park with Max, a little girl ran up to him. She looked at Max and shouted, "That’s my dog!" Tim felt sad. The girl told him that Max’s real name was Buddy, and he had run away from home about a week ago. Even though Tim didn’t want to let Max go, he knew it was the right thing to do. So, the next day, Tim and his parents took Max to the girl’s house, and Tim said goodbye.
It was hard to say goodbye, but Tim was proud that he did the right thing. A few days later, Tim’s parents surprised him by adopting a new puppy from the animal shelter. Tim named the new dog Lucky, and even though he missed Max, he was happy to have another dog to play with.
"""
tokens = nltk.word_tokenize(text.lower())

In [ ]:
detect_errors(text1)

Transposition Errors Found:
Letter transposition detected: 'nosie' might be noise
Letter transposition detected: 'littel' might be little


Vowel Substitution Errors Found:
'tim' might have a vowel substitution. Suggestions: {'tum', 'tam'}
'coller' might have a vowel substitution. Suggestions: {'culler', 'caller', 'collar'}
'thot' might have a vowel substitution. Suggestions: {'that'}
'max' might have a vowel substitution. Suggestions: {'mix', 'mux'}
'sed' might have a vowel substitution. Suggestions: {'sad', 'sod', 'sud'}
'animel' might have a vowel substitution. Suggestions: {'animal'}
'sheltur' might have a vowel substitution. Suggestions: {'shelter'}


Omission Errors Found:
'tim' might have a letter omitted. Suggestions: {'trim', 'time', 'stim'}
'nosie' might have a letter omitted. Suggestions: {'nosine'}
'hideing' might have a letter omitted. Suggestions: {'hideling'}
'coller' might have a letter omitted. Suggestions: {'collier', 'choller', 'collery'}
'thot' might have a letter o